In [1]:
import numpy as np
import pandas as pd
import random
from pathlib import Path
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, TensorDataset
from sklearn.preprocessing import MinMaxScaler

# 난수 생성 고정
RANDOM_SEED = 42
np.random.seed(RANDOM_SEED)
random.seed(RANDOM_SEED)
torch.manual_seed(RANDOM_SEED)

class TimeSeriesVAE(nn.Module):
    def __init__(self, num_features=52, latent_dim=20, hidden_dims=[32, 64]):
        super(TimeSeriesVAE, self).__init__()
        
        self.num_features = num_features
        self.latent_dim = latent_dim
        self.hidden_dims = hidden_dims
        
        # 인코더
        encoder_layers = []
        in_channels = num_features
        
        for h_dim in hidden_dims:
            encoder_layers.extend([
                nn.Conv1d(in_channels, h_dim, kernel_size=3, stride=1, padding=1),
                nn.BatchNorm1d(h_dim),
                nn.LeakyReLU(0.2),
                nn.Dropout(0.2)
            ])
            in_channels = h_dim
            
        self.encoder = nn.Sequential(*encoder_layers)
        
        # 디코더
        decoder_layers = []
        hidden_dims.reverse()
        
        for i in range(len(hidden_dims) - 1):
            decoder_layers.extend([
                nn.ConvTranspose1d(hidden_dims[i], hidden_dims[i + 1], 
                                 kernel_size=3, stride=1, padding=1),
                nn.BatchNorm1d(hidden_dims[i + 1]),
                nn.LeakyReLU(0.2),
                nn.Dropout(0.2)
            ])
            
        decoder_layers.extend([
            nn.ConvTranspose1d(hidden_dims[-1], num_features, 
                             kernel_size=3, stride=1, padding=1),
            nn.Tanh()
        ])
        
        self.decoder = nn.Sequential(*decoder_layers)
        
        # 중간 레이어들은 forward pass에서 동적으로 초기화됨
        self.fc_mu = None
        self.fc_logvar = None
        self.fc_decode = None

    def _get_conv_output_size(self, seq_length):
        x = torch.zeros(1, self.num_features, seq_length)
        x = self.encoder(x)
        return x.numel() // x.shape[0]

    def initialize_layers(self, seq_length):
        conv_output_size = self._get_conv_output_size(seq_length)
        
        self.fc_mu = nn.Linear(conv_output_size, self.latent_dim)
        self.fc_logvar = nn.Linear(conv_output_size, self.latent_dim)
        self.fc_decode = nn.Linear(self.latent_dim, conv_output_size)

    def encode(self, x):
        if self.fc_mu is None:
            self.initialize_layers(x.shape[2])
            
        h = self.encoder(x)
        h = h.reshape(h.size(0), -1)  # Changed from view to reshape
        
        return self.fc_mu(h), self.fc_logvar(h)
    
    def reparameterize(self, mu, logvar):
        if self.training:
            std = torch.exp(0.5 * logvar)
            eps = torch.randn_like(std)
            return mu + eps * std
        return mu
    
    def decode(self, z):
        h = self.fc_decode(z)
        h = h.reshape(h.size(0), self.hidden_dims[-1], -1)  # Changed from view to reshape
        return self.decoder(h)

    def forward(self, x):
        mu, logvar = self.encode(x)
        z = self.reparameterize(mu, logvar)
        return self.decode(z), mu, logvar

def prepare_data(data_path, seq_length_train, seq_length_test, num_features):
    """데이터 전처리 함수"""
    # 데이터 로드
    train_data = pd.read_csv(data_path / "train.csv")
    test_data = pd.read_csv(data_path / "test.csv")
    
    # 필요 피처 선택
    non_numeric_cols = ["faultNumber", "simulationRun", "sample"]
    use_cols = train_data.columns.difference(non_numeric_cols)
    
    X_train = train_data[use_cols]
    X_test = test_data[use_cols]
    
    # 데이터 스케일링
    scaler = MinMaxScaler(feature_range=(-1, 1))
    X_train_scaled = scaler.fit_transform(X_train)
    X_test_scaled = scaler.transform(X_test)
    
    # PyTorch Tensor로 변환
    X_train_tensor = torch.FloatTensor(X_train_scaled)
    X_test_tensor = torch.FloatTensor(X_test_scaled)
    
    # 시계열 시퀀스로 재구성
    def reshape_sequences(data, seq_length):
        # 데이터를 연속적인 메모리 블록으로 만듦
        data = data.contiguous()
        
        total_sequences = len(data) // seq_length
        if total_sequences == 0:
            raise ValueError(f"Sequence length {seq_length} is too long for data with length {len(data)}")
        
        useful_data_len = total_sequences * seq_length
        reshaped_data = data[:useful_data_len].reshape(total_sequences, num_features, seq_length)
        return reshaped_data
    
    try:
        X_train_sequences = reshape_sequences(X_train_tensor, seq_length_train)
        X_test_sequences = reshape_sequences(X_test_tensor, seq_length_test)
        
        print(f"Training sequences shape: {X_train_sequences.shape}")
        print(f"Test sequences shape: {X_test_sequences.shape}")
        
        return X_train_sequences, X_test_sequences
    
    except ValueError as e:
        print(f"Error in sequence reshaping: {e}")
        return None, None

def loss_function(recon_x, x, mu, logvar, beta=0.1):
    """베타-VAE 손실 함수"""
    recon_loss = nn.MSELoss(reduction='mean')(recon_x, x)
    kld_loss = -0.5 * torch.mean(1 + logvar - mu.pow(2) - logvar.exp())
    return recon_loss + beta * kld_loss

def train_model(model, train_loader, num_epochs, device, beta=0.1):
    optimizer = optim.AdamW(model.parameters(), lr=1e-3, weight_decay=1e-5)
    scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, mode='min', factor=0.5, patience=3, verbose=True)
    
    model.train()
    train_losses = []
    
    for epoch in range(num_epochs):
        epoch_loss = 0
        for batch_idx, (batch_data,) in enumerate(train_loader):
            batch_data = batch_data.to(device)
            optimizer.zero_grad()
            
            recon_batch, mu, logvar = model(batch_data)
            loss = loss_function(recon_batch, batch_data, mu, logvar, beta)
            
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), max_norm=1.0)
            optimizer.step()
            
            epoch_loss += loss.item()
            
        avg_loss = epoch_loss / len(train_loader)
        train_losses.append(avg_loss)
        scheduler.step(avg_loss)
        
        print(f'Epoch {epoch+1}/{num_epochs}, Average Loss: {avg_loss:.6f}')
    
    return model, train_losses

def detect_anomalies(model, data, threshold_percentile=95):
    model.eval()
    reconstruction_errors = []
    
    with torch.no_grad():
        for batch in DataLoader(TensorDataset(data), batch_size=32):
            batch = batch[0].to(next(model.parameters()).device)
            recon, _, _ = model(batch)
            error = torch.mean((recon - batch) ** 2, dim=(1, 2))
            reconstruction_errors.extend(error.cpu().numpy())
    
    threshold = np.percentile(reconstruction_errors, threshold_percentile)
    is_anomaly = np.array(reconstruction_errors) > threshold
    
    return is_anomaly, reconstruction_errors, threshold

def main():
    # 설정
    DATA_PATH = Path("./data")
    seq_length_train = 500  # Reduced from 500
    seq_length_test = 960   # Reduced from 960
    num_features = 52
    batch_size = 32
    num_epochs = 50
    
    # GPU 설정
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    print(f"Using device: {device}")
    
    # 데이터 준비
    X_train_tensor, X_test_tensor = prepare_data(DATA_PATH, 
                                               seq_length_train, 
                                               seq_length_test, 
                                               num_features)
    
    if X_train_tensor is not None and X_test_tensor is not None:
        # 데이터를 device로 이동
        X_train_tensor = X_train_tensor.to(device)
        X_test_tensor = X_test_tensor.to(device)
        
        # 모델 초기화
        model = TimeSeriesVAE(num_features=num_features, 
                            latent_dim=20, 
                            hidden_dims=[32, 64]).to(device)
        
        # 데이터로더 설정
        train_loader = DataLoader(TensorDataset(X_train_tensor), 
                                batch_size=batch_size, 
                                shuffle=True)
        
        # 모델 학습
        model, train_losses = train_model(model, train_loader, 
                                        num_epochs=num_epochs, 
                                        device=device)
        
        # 이상 탐지 수행
        anomalies, errors, threshold = detect_anomalies(model, X_test_tensor)
        print(f"Detected {sum(anomalies)} anomalies out of {len(anomalies)} samples")
        print(f"Anomaly threshold: {threshold:.6f}")
        print(f"Mean reconstruction error: {np.mean(errors):.6f}")
        
        return model, anomalies, errors, threshold
    else:
        print("데이터 준비 과정에서 오류가 발생했습니다.")
        return None, None, None, None

if __name__ == "__main__":
    main()

Using device: cuda
Training sequences shape: torch.Size([500, 52, 500])
Test sequences shape: torch.Size([740, 52, 960])


/data/ephemeral/home/.conda/lib/python3.10/site-packages/torch/optim/lr_scheduler.py:62: UserWarning: The verbose parameter is deprecated. Please use get_last_lr() to access the learning rate.
  warnings.warn(


RuntimeError: Input type (torch.FloatTensor) and weight type (torch.cuda.FloatTensor) should be the same or input should be a MKLDNN tensor and weight is a dense tensor